In [16]:
import pandas as pd
import json
import gspread
import os
from dotenv import load_dotenv

# Carrega variáveis de ambiente com load_dotenv
load_dotenv()
KEY_JSON = os.getenv('KEY_JSON')

# Autenticação com Google Sheets e configuração de acesso a planilha
google_client = gspread.service_account_from_dict(json.loads(KEY_JSON))
SHEET_ID = '1tbJh_GQol0Ax5uB-AjwI26yW9piCIp0peuKV1N5nMeQ'
worksheet = google_client.open_by_key(SHEET_ID).worksheet('db')

# Carregar dados com pandas.clipboard()
df_registros = pd.read_clipboard()

In [17]:
# Função para enviar dados em batch
def enviar_dados_em_batch(dados):
    batch_requests = []

    for index, registro in dados.iterrows():
        # Verifica se todos os campos estão preenchidos e se "Quantidade Remessas" é maior que 0
        if all(pd.notna(registro[1:])) and registro['Quantidade Remessas'] > 0:
            # Adicionar a linha ao batch request
            linha_dados = [
                registro['Unidade'], 
                registro['Motorista'], 
                registro['Placa'], 
                registro['Chegada CD'], 
                registro['Início do Carregamento'], 
                registro['Fim do Carregamento'], 
                registro['Quantidade Remessas']
            ]
            batch_requests.append(linha_dados)

    # Enviar os dados em lote
    if batch_requests:
        worksheet.append_rows(batch_requests)
        print(f"{len(batch_requests)} registros enviados para o Google Sheets com sucesso!")
    else:
        print("Nenhum dado válido para enviar.")

# Testar o envio em batch requests
if not df_registros.empty:
    enviar_dados_em_batch(df_registros)
else:
    print("Nenhum dado encontrado para enviar.")

41 registros enviados para o Google Sheets com sucesso!


In [18]:
# Apaga os dados do intervalo especificado
worksheet.batch_clear(['A2:G10000'])

{'spreadsheetId': '1tbJh_GQol0Ax5uB-AjwI26yW9piCIp0peuKV1N5nMeQ',
 'clearedRanges': ['db!A2:G10000']}